<a id='initialisation'></a>
___
___
## <font color='coral'>**Appel de l'environment** </font> [▲](#sommaire)

In [1]:
%reset -f
import ipywidgets as widgets
%matplotlib widget
from ipywidgets import Layout, interact
import matplotlib.pyplot as plt
from RVutils import mydict, hdf5_to_dict, Fourier2D
import numpy as np

## Set up rc
plt.rcParams['image.cmap'] = 'RdYlBu_r'
plt.rcParams['axes.formatter.limits'] = (-3,1)
plt.rcParams['lines.linewidth'] = 2.5
plt.rcParams.update({"axes.grid" : True})
plt.rcParams['lines.markersize'] = 8
plt.rcParams['lines.markeredgecolor'] = 'k'
plt.rcParams['lines.markeredgewidth'] = 2.0
plt.rc('font',size=18)
me = (0.1,0.1) #Markevery

## <font color='coral'>**Chargement des données** </font> 

In [6]:
q05_spectra = hdf5_to_dict("data/q05_growthrate_2Dspectra.hdf5")
q15_spectra = hdf5_to_dict("data/q15_growthrate_2Dspectra.hdf5")
print(q05_spectra.keys())

dict_keys(['Phithphi', 'T_rpeak', 'Tir', 'growth_rate_spectra', 'm', 'n', 'phig', 'q_rpeak', 'rpeak', 'thetag', 'time'])


## <font color='royalblue'>**Tracé du spectre de $\phi^2$** </font> 

In [3]:
## Create custom dictionnary class for spectra
class mydict2(dict):
    def __init__(self, *args, **kwargs):
        super(mydict2, self).__init__(*args, **kwargs)
        self.__dict__ = self
        
        self['Nphi'] = len(self['phig'])-1
        self['Ntheta'] = len(self['thetag'])-1
        self['Phithphisq'] = self['Phithphi']**2

        ## Evolution of mn spectra
        tevol_TFPhi2D = np.zeros((len(self['time']),self['Nphi'],self['Ntheta']))
        for it in range(len(self['time'])):
            tevol_TFPhi2D[it,:,:] = np.abs(Fourier2D(self['Phithphisq'][it,0:self['Nphi']+1,0:self['Ntheta']+1],self['phig'],self['thetag'])[0])
        self['TFPhi2D'] = tevol_TFPhi2D

        ## Coordinates in m,n space
        m2d,n2d = Fourier2D(self['Phithphisq'][-1,0:self['Nphi']+1,0:self['Ntheta']+1],self['phig'],self['thetag'])[1:]
        dm2d = (m2d[1]-m2d[0])//2 ; m2d = m2d-dm2d
        dn2d = (n2d[1]-n2d[0])//2 ; n2d = n2d-dn2d
        self['m2d'] = m2d 
        self['n2d'] = n2d 

    def __getitem__(self, key): 
        if  key     == 'Nphi'       : return len(self['phig'])-1
        if  key     == 'Ntheta'     : return len(self['thetag'])-1
        if  key     == 'Phithphisq' : return self['Phithphi']**2
        else: return super().__getitem__(key)

    def estimate_GRlin2D(self,tinit,tend):
        itinit = np.argmin(np.abs(self['time']-tinit))
        itend  = np.argmin(np.abs(self['time']-tend))
        Gammalin2D = np.log(self['TFPhi2D'][itend,:,:]/self['TFPhi2D'][itinit,:,:])/(tend-tinit)
        return Gammalin2D

## end of mydict2

def hdf5_to_dict2(filename, cls=mydict2):
    import h5py
    """Load a dictionary of arrays and strings as unicode characters from an HDF5 file."""
    with h5py.File(filename, 'r') as f:
        d = {}
        for k in f.keys():
            v = f[k][()]
            if isinstance(v, bytes):
                d[k] = v.decode('utf-8')
            else:
                d[k] = np.array(v)
    return cls(d)

q05_spectra = hdf5_to_dict("data/q05_growthrate_2Dspectra.hdf5",cls=mydict2)
q15_spectra = hdf5_to_dict("data/q15_growthrate_2Dspectra.hdf5",cls=mydict2)

for iq,q in enumerate([q05_spectra,q15_spectra]):
    print('qrpeak = %.2f' % q['q_rpeak'])

    tevol_TFPhi2D = q['TFPhi2D']
    m2d,n2d = q['m2d'],q['n2d']
    Nphi = q['Nphi']
    Ntheta = q['Ntheta']

    ## Calculate the linear growth rate between tinit and tend
    tinit = 15e3
    tend  = 19e3

    Gammalin2D = q.estimate_GRlin2D(tinit,tend)
    
    C = 140
    n_ex = np.argmin(np.abs(n2d-dn2d-C))
    m_ex = np.argmin(np.abs(m2d-dm2d+C*q['q_rpeak']))

    fig = plt.figure(figsize=(18,6))

    ## Plot the 2D spectrum
    ax1 = fig.add_subplot(131)
    p1 = ax1.pcolormesh(m2d,n2d,np.log10(tevol_TFPhi2D[-30,:,:]),vmin=-10,vmax=0)    
    fig.colorbar(p1)
    ax1.set_xlabel(r'$m$')
    ax1.set_ylabel(r'$n$')
    ax1.set_title(r'$\log_{10}(|\tilde{\phi}|^2)$')
    ax1.plot(m2d,-m2d/q['q_rpeak'],c='k',ls='--')
    ## Add the point n_ex,m_ex to the plot
    ax1.plot(m2d[m_ex],n2d[n_ex],'r+')

    ## Plot the time evolution of the spectrum at n_ex,m_ex
    ax2 = fig.add_subplot(132)
    ax2.plot(q['time'],np.log10(tevol_TFPhi2D[:,n_ex,m_ex]))
    ax2.axvline(tinit,c='k',ls='--')
    ax2.axvline(tend,c='k',ls='--')

    ## Plot the growth rate
    ax3 = fig.add_subplot(133)
    p3 = ax3.pcolormesh(m2d,n2d,Gammalin2D[:,:],vmin=0) 
    fig.colorbar(p3)

    fig.tight_layout()

qrpeak = 0.96


E:\Temp\ipykernel_5936\1795303237.py:33: RuntimeWarning: invalid value encountered in true_divide
  Gammalin2D = np.log(self['TFPhi2D'][itend,:,:]/self['TFPhi2D'][itinit,:,:])/(tend-tinit)


NameError: name 'dn2d' is not defined